## Práctica de Minería de Datos.

#### Realizada por Araceli Macía Barrado

##### Segunda Parte
----------

El objetivo de esta práctica es poder identificar a partir de la letra de la canciones el genero "Reggueaton".
Dado que el texto de las canciones de este genero es muy identificativo, por el contenido sexual, referencias a alcohol o drogas o a armas de fuego; se deberia poder identificar por la letra cuando una cancion pertenece a este genero o no. 
<br>

Dificultades encontradas: 
1. El lenguaje de las canciones Reggeaton, es una mezcla entre español-latino y Spanglish.
2. Aunque parezca mentira, el Reggeaton evoluciona para mostrarse menos incorrecto.


<br>
La práctica la he dividido en los siguientes NoteBook:
1. PT_MinDatos_Reg1 :  Código para la recuperacion de los textos de las canciones.
2. PT_MinDatos_Reg2 : Código para tratamiento y clasificación de textos.


----------------

### Recuperación y Tratamiento de Datos.

Para realizar la practica he descargado las letras de las canciones de la página : 
* http://www.sonicomusica.com

Previo al ejercicio, he realizado consultas para seleccionar dos ficheros de canciones, una de canciones típicas de Reggeaton, y otra de canciones que no lo son. A partir de estos ficheros, que contienen los enlaces a las canciones, procederé a realizar la descarga de las canciones. Cada canción se descarga en un fichero, y el texto se guardará tokenizado. A su vez, los ficheros los he separado en dos directorios, para tener separadas las letras de las canciones de Reggeaton de las que no lo son.


In [1]:
#Importacion de librerias necesarias.
import requests
import lxml
import numpy as np
import pandas as pd
import nltk
from lxml import html
import time
from os import listdir
import fnmatch



In [2]:
#Reviso que estan los directorios.
% ls FicDatos/    

Canciones_NR.txt  NO_REG/           REG/              canciones_RG.txt


In [3]:
def words_file(ruta, fic):   #devolver las palabras de cada fichero.
    ficLeido=open(ruta+fic, 'r')
    try:
        texto=ficLeido.read().decode('utf-8')
        tokens =nltk.word_tokenize(texto)         #tokens=['a','b','c'] lo dejo como unicode.
        ficLeido.close()
        return tokens
    except UnicodeDecodeError:     
        print "error en", ruta+fic
        return ["---"]
   


In [4]:
#funcion para leer los ficheros del directorio.
def ficheros(ruta2,pa):
    tot=[] 
    for fileid in listdir(ruta2):
        if fnmatch.fnmatch(fileid, pa):
             tot.append(fileid)
    return tot



In [5]:
def words_all_file(rutas):   #devolver las palabras de los ficheros. Devuelve una lista.
    tokens=[]
    
    for t in rutas:
        ruta1=t[0]
        patron1=t[1]
       
        for fileid in ficheros(ruta1,patron1):
            ficLeido1=open(ruta1+fileid, 'r')
            try:
                texto=ficLeido1.read().decode('utf-8')                
                tokens.extend(nltk.word_tokenize(texto))        #tokens=['a','b','c'] UNICODE
                ficLeido1.close()
            except UnicodeDecodeError:     
                print "error en", ruta+fic
                return ["---"]
    return tokens    
    

In [6]:
def document_features(document): 
# de cada documento, que contine la lista de palabras, devuelvo si contiene o no las palabras encontradas filtradas.
        
    document_words = set(document) 
    features = {}
    for word in word_features:
        w1=word.encode('utf-8')  #convierto a utf8 para poder formatearlo.
        features['contains({})'.format(w1)] = (word in document_words)  
    return features
   

In [7]:
ruta1="FicDatos/REG/"
ruta2="FicDatos/NO_REG/"

#creo documento de tipos list:   (tokensCANCION,"REG")
documents1 = [ (list(words_file(ruta1,fileid)), "REG")
              for fileid in ficheros(ruta1,"REG*.txt")]

#creo documento de tipos list:   (tokensCANCION,"NO REG")

documents2 = [ (list(words_file(ruta2,fileid)), "NO_REG")
              for fileid in ficheros(ruta2,"NR*.txt")]

#Uno las listas, de forma que ahora tengo una super lista, donde cada elemento son las palabras de una cancion
#y un indicador de si es Regueton o no lo es.
documents= documents1 + documents2
np.random.shuffle(documents)

    

In [8]:
rutas=[("FicDatos/REG/","REG*.txt"),("FicDatos/NO_REG/","NR*.txt")]
allwordsT=words_all_file(rutas) #recupero todas las palabras de los ficheros.
print "numero de palabras total", len(allwordsT)  #Numero de palabras en total de todas las canciones.

numero de palabras total 104261


In [9]:
all_words = nltk.FreqDist(w.lower() for w in allwordsT) #frecuencia de cada palabra, de todos los documentos.
print "Numero de palabras total", len(allwordsT)  #Numero de palabras en total de todas las canciones.
word_features = list(all_words)[:3000]  #Me quedo con las 3000 palabras de la lista
print "numero de palabras sin repetir", len(all_words)


Numero de palabras total 104261
numero de palabras sin repetir 9179


In [10]:
for w, frequency in all_words.most_common(10):
    print(u'{};{}'.format(w, frequency))

,;4476
que;4046
y;2677
la;2535
no;2176
me;1998
de;1983
te;1920
el;1852
a;1783


In [11]:
featuresets = [(document_features(d), c) for (d,c) in documents]
train_set, test_set = featuresets[100:], featuresets[:100]


In [12]:
print len(featuresets)
print len (train_set)
print len (test_set)

280
180
100


In [13]:
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [14]:
print(nltk.classify.accuracy(classifier, test_set))  #Tiene un acierto bastante alto

0.85


In [15]:
cpdist = classifier._feature_probdist
        
print('Caracteristicas mas Representativas: ')

for (fname, fval) in classifier.most_informative_features(15):
    def labelprob(l):
        return cpdist[l, fname].prob(fval)

    labels = sorted([l for l in classifier._labels
                     if fval in cpdist[l, fname].samples()],
                    key=labelprob)
    if len(labels) == 1:
        continue
    l0 = labels[0]
    l1 = labels[-1]
    if cpdist[l0, fname].prob(fval) == 0:
        ratio = 'INF'
    else:
        ratio = '%8.1f' % (cpdist[l1, fname].prob(fval) /
                           cpdist[l0, fname].prob(fval))
    print(('%24s = %-14r %6s : %-6s = %s : 1.0' %
           (fname, fval, ("%s" % l1)[:6], ("%s" % l0)[:6], ratio)))

Caracteristicas mas Representativas: 
           contains(the) = True              REG : NO_REG =      9.7 : 1.0
            contains(x2) = True              REG : NO_REG =      9.4 : 1.0
         contains(llego) = True              REG : NO_REG =      7.7 : 1.0
         contains(llama) = True              REG : NO_REG =      6.6 : 1.0
           contains(fin) = True           NO_REG : REG    =      6.6 : 1.0
             contains(:) = True           NO_REG : REG    =      6.6 : 1.0
           contains(has) = True           NO_REG : REG    =      6.2 : 1.0
             contains(2) = True              REG : NO_REG =      5.7 : 1.0
            contains(ve) = True              REG : NO_REG =      5.7 : 1.0
      contains(silencio) = True           NO_REG : REG    =      5.7 : 1.0
       contains(sentido) = True           NO_REG : REG    =      5.0 : 1.0
         contains(mucha) = True              REG : NO_REG =      5.0 : 1.0
          contains(rico) = True              REG : NO_REG =   

----------------

Hasta aqui, no he realizado ningun tipo de filtrado en los datos de partida, ahora voy a filtrar para quitar caracteres no alfanumericos, y tambien voy a dejar de considerar las palabras que no esten en el diccionario de Español, a ver como reacciona el clasificador.

In [16]:
from nltk.corpus import stopwords 
stop=stopwords.words("spanish")
Spanish_vocab = set(w.lower() for w in nltk.corpus.cess_esp.words() )

def words_all_file_SinCaraSTOPDIC(rutas):   #devolver las palabras de los ficheros. Devuelve una lista.
    tokens=[]
    
    for t in rutas:
        ruta1=t[0]
        patron1=t[1]
       
        for fileid in ficheros(ruta1,patron1):
            ficLeido1=open(ruta1+fileid, 'r')
            try:
                texto=ficLeido1.read().decode('utf-8')
                t1=nltk.word_tokenize(texto)              
                for w in t1:  
                    wo=w.lower()
                    if wo.isalpha():  #Solo considero palabras alfanumericas.
                        if wo not in stop:  #solo considero las palabras que no estan en stopwords.
                            if wo in Spanish_vocab:  #solo las que estan en el diccionarios Español
                                #wo=wo.encode('utf-8') 
                                tokens.append(wo)                            
                ficLeido1.close()
            except UnicodeDecodeError:     
                print "error en", ruta+fic
                return ["---"]
   
    return tokens   

In [17]:
rutas=[("FicDatos/REG/","REG*.txt"),("FicDatos/NO_REG/","NR*.txt")]

allwordsSIN=words_all_file_SinCaraSTOPDIC(rutas) #recupero todas las palabras de los ficheros.

print "Numero de palabras total", len(allwordsSIN)  #Numero de palabras en total de todas las canciones.

all_words = nltk.FreqDist(w.lower() for w in allwordsSIN) #frecuencia de cada palabra, de todos los documentos.
word_features = list(all_words)[:3000]  #Me quedo con las 3000 palabras de la lista
print "numero de palabras sin repetir", len(all_words)



Numero de palabras total 27553
numero de palabras sin repetir 2972


In [18]:
featuresets = [(document_features(d), c) for (d,c) in documents]
train_set, test_set = featuresets[100:], featuresets[:100]

In [19]:
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))  #Tiene un acierto superior al anterior.
#classifier.show_most_informative_features(10) #las 10 palabras mas identificativas 

0.92


In [20]:
cpdist = classifier._feature_probdist
        
print('Caracteristicas mas Representativas: ')

for (fname, fval) in classifier.most_informative_features(15):
    def labelprob(l):
        return cpdist[l, fname].prob(fval)

    labels = sorted([l for l in classifier._labels
                     if fval in cpdist[l, fname].samples()],
                    key=labelprob)
    if len(labels) == 1:
        continue
    l0 = labels[0]
    l1 = labels[-1]
    if cpdist[l0, fname].prob(fval) == 0:
        ratio = 'INF'
    else:
        ratio = '%8.1f' % (cpdist[l1, fname].prob(fval) /
                           cpdist[l0, fname].prob(fval))
    print(('%24s = %-14r %6s : %-6s = %s : 1.0' %
           (fname, fval, ("%s" % l1)[:6], ("%s" % l0)[:6], ratio)))

Caracteristicas mas Representativas: 
          contains(duro) = True              REG : NO_REG =     11.0 : 1.0
          contains(vida) = True           NO_REG : REG    =      9.2 : 1.0
          contains(alma) = True           NO_REG : REG    =      9.0 : 1.0
          contains(ropa) = True              REG : NO_REG =      7.7 : 1.0
      contains(haciendo) = True              REG : NO_REG =      7.7 : 1.0
           contains(par) = True              REG : NO_REG =      7.7 : 1.0
      contains(corazón) = True           NO_REG : REG    =      6.7 : 1.0
           contains(fin) = True           NO_REG : REG    =      6.6 : 1.0
        contains(hombre) = True              REG : NO_REG =      6.6 : 1.0
         contains(llama) = True              REG : NO_REG =      6.6 : 1.0
         contains(ritmo) = True              REG : NO_REG =      6.3 : 1.0
            contains(pa) = True              REG : NO_REG =      6.3 : 1.0
         contains(gusta) = True              REG : NO_REG =    

-------------------------
## Probando el clasificador

In [21]:
#Vamos a probar con una cancion de Alex Ubago.
cancion="Si ayer tuviste un dia gris, tranquila, yo haré canciones para ver si así consigo acerte sonreir, si \
lo q quieres es huir, camina, yo haré canciones para ver, si así consigo fuerzas pa' vivir... No tengo mas motivos\
para darte que este miedo que me dá, el no volver a verte, nunca más... Creo ver la lluvia caer en mi ventana te veo \
pero no está lloviendo no es más que un reflejo de mi pensamiento, hoy te echo de menos... Yo sólo quiero hacerte \
saber amiga estes donde estes que si te falta el aliento yo te lo daré, y si te sientes sóla hablame, que te estaré \
escuchando aunque no te pueda ver... aunque no te pueda ver... De tantas cosas que perdí diría que sólo guardo lo que \
fue magico tiempo que nació en abril, miradas tristes sobre mi se anidan y se hacen parte de mi ser y ahora siempre \
llueve por que estoy sin ti... para darte que esta fría soledad, que necesito darte tantas cosas más... Creo ver la \
lluvia caer en mi ventana te veo pero no está lloviendo no es mas que un reflejo de mi pensamiento hoy te echo de \
menos... Yo sólo quiero hacer saber amiga estes donde estes que si te falta aliento yo te lo daré..."

In [22]:
cancionSubago=nltk.word_tokenize(cancion.decode('utf-8'))

In [23]:
classifier.classify(document_features(cancionSubago))

'NO_REG'

In [24]:
letra2="ella no ta' enamora' de mi Yo tampoco Pero le gusta como yo le doy Yo la pongo a volar Cuando yo \
le doy besos Pero no ta' enamora' de mi Ella solamente quiere pasar un momento lunático \
She said she's not in love with me, no! No! But she loves how I give it to her. Yes! Yes! Sexy! \
And I blow her mind! Every time that I kiss her But she said she's not in love with me. \
Yeah right! Ella siempre me llama a las 3AM Dice que soy su pana El que le quita las ganas \
Ha ha ha! Y si ella supiera que me llama su hermana Porque tambien soy su pana \
El que le quita las ganasWho the hell is this? Texting me at 3:46AM It must be a trick \
And if she texting me at this time She's looking for a Richard Or should I say Dick \
Mami yo te doy pao! pao! Ella dice que yo soy mal hablao \
No mamita yo no soy mal hablao Que yo te hablo directo de Miami al Cibao So deja el papelazo \
Para los turistas y los payasos Hablame claro que tu sabes bien Que te gusta lo malo Dale palo!  \
Yo te doy caramelo Pero deja los celos Ella dice que no esta enamorada de mi \
Eso me conviene Don Miguelo Ella no ta' enamora' de mi no! no! \
Pero le gusta como yo le doy. si! si! Yo la pongo a volar Cuando yo le doy besos \
Pero no ta' enamora' de mi no! no! She said she's not in love with me, no! No! \
But she loves how I give it to her. Yes! Yes! Sexy! And I blow her mind! \
Every time that I kiss her But she said she's not in love with me. Yeah right. \
Casi siempre me llama a las 3AM Dice que soy su pana El que le quita las ganas \
Yo nunca la dejo a medias Tu entiende el drama Dice que soy su pana \
El que le quita las ganas Hemos hecho de to' Muchas posiciones en la escalera \
Amarrao en la cama Yo encima de ella Amanecimos en la bañera \
Y me la comi embarra de nutella Son requisitos que otros no nan usado \
Y ella solo dice wow Lo que yo hago nunca se ha inventado \
Porque ni el kamasutra Se lo ha inventado Oiga! \
Ella no ta' enamora' de mi no! no! Pero le gusta como yo le doy. si! si! \
Yo la pongo a volar Cuando yo le doy besos Pero no ta' enamora' de mi no! no! "

In [25]:
cancionR=nltk.word_tokenize(letra2.decode('utf-8'))
classifier.classify(document_features(cancionR))

'REG'

In [26]:
classifier.prob_classify(document_features(cancionR)).max()

'REG'